In [3]:
import pickle
import os

from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


In [4]:
df = pd.read_csv('../../data/Top100_prediction_con.csv')

In [5]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'BOS_3') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'sales', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100),
        lags=14
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)
    
    # Predecir
    predicciones = forecaster.predict(steps=30, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    print(f"Predicciones generadas para el producto: {producto}")


Predicciones generadas para el producto: ACCESORIES_1_108
Predicciones generadas para el producto: HOME_&_GARDEN_1_027
Predicciones generadas para el producto: HOME_&_GARDEN_1_053
Predicciones generadas para el producto: HOME_&_GARDEN_1_140


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


Predicciones generadas para el producto: HOME_&_GARDEN_1_177
Predicciones generadas para el producto: HOME_&_GARDEN_1_272
Predicciones generadas para el producto: HOME_&_GARDEN_1_344
Predicciones generadas para el producto: HOME_&_GARDEN_1_373
Predicciones generadas para el producto: HOME_&_GARDEN_1_440
Predicciones generadas para el producto: HOME_&_GARDEN_1_537


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


Predicciones generadas para el producto: SUPERMARKET_1_096
Predicciones generadas para el producto: SUPERMARKET_2_019


...

/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


Predicciones generadas para el producto: ACCESORIES_1_158
Predicciones generadas para el producto: SUPERMARKET_3_282


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


Predicciones generadas para el producto: ACCESORIES_1_354
Predicciones generadas para el producto: HOME_&_GARDEN_1_106


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


In [6]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediccion'] = tiendas_inventario['prediccion'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['producto','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,producto,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,10,15.0,0
1,HOME_&_GARDEN_1_027,59,74.0,0
2,HOME_&_GARDEN_1_053,120,81.0,0
3,HOME_&_GARDEN_1_140,46,41.0,0
4,HOME_&_GARDEN_1_177,36,59.0,0
...,...,...,...,...
95,SUPERMARKET_3_499,273,313.0,0
96,ACCESORIES_1_158,630,579.0,0
97,SUPERMARKET_3_282,326,316.0,0
98,ACCESORIES_1_354,624,687.0,0


In [7]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

25899
27633.0
-1734.0


In [ ]:
tienda =  pd.DataFrame(resultados)
file_name = "BOS_3_unidades.xlsx"
tienda.to_excel(file_name)

## PREDICCIONES

In [6]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())  

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'NYC_3') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'sales', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100),
        lags=7
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)

    # Crear un rango de fechas y el dataframe de exógenas
    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    date_range = pd.date_range(start=start_date, end=end_date)
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)
    
    # Predecir
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Crear un diccionario con claves 'dia1', 'dia2', ..., 'dia60'
    predicciones_dict = {f'dia{i+1}': pred for i, pred in enumerate(predicciones.values)}
    
    # Almacenar los resultados en un diccionario para cada producto
    resultados.append({
        'producto': producto,
        **predicciones_dict  # Desempaquetar el diccionario de predicciones
    })

# Convertir la lista de resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Mostrar el DataFrame final con las columnas dia1, dia2, ..., dia60
print(df_resultados)


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 

               producto       dia1       dia2       dia3       dia4  \
0      ACCESORIES_1_108   1.452962   0.750564   0.735587   0.938167   
1   HOME_&_GARDEN_1_027   4.474940   5.130734   4.119028   4.172108   
2   HOME_&_GARDEN_1_053   7.814837  12.020758   7.439361   7.424397   
3   HOME_&_GARDEN_1_140   4.652560   5.025778   4.002103   3.569119   
4   HOME_&_GARDEN_1_177   5.669504   5.836857   3.855762   4.415171   
..                  ...        ...        ...        ...        ...   
95    SUPERMARKET_3_499  12.305567  10.023282  12.284995   5.778226   
96     ACCESORIES_1_158   5.519582   6.620786   5.095683   4.566849   
97    SUPERMARKET_3_282  55.423673  71.586254  57.270722  50.600475   
98     ACCESORIES_1_354   4.417852   9.111879   6.381964   5.877536   
99  HOME_&_GARDEN_1_106  19.772615  19.318091  16.151111  12.067254   

         dia5       dia6       dia7       dia8       dia9  ...      dia51  \
0    0.967821   0.640012   1.092960   0.850230   0.751416  ...   0.912

In [7]:
df_resultados

,producto,dia1,dia2,dia3,dia4,dia5,dia6,dia7,dia8,dia9,...,dia51,dia52,dia53,dia54,dia55,dia56,dia57,dia58,dia59,dia60
0,ACCESORIES_1_108,1.452962,0.750564,0.735587,0.938167,0.967821,0.640012,1.092960,0.850230,0.751416,...,0.912376,1.238464,1.129545,0.374548,0.108536,0.662047,1.080099,0.912376,1.238464,1.129545
1,HOME_&_GARDEN_1_027,4.474940,5.130734,4.119028,4.172108,5.040788,4.110226,4.551832,5.481818,6.263417,...,8.649107,7.120127,6.851071,7.349258,6.536507,6.103409,7.134370,8.286472,6.436591,6.970675
2,HOME_&_GARDEN_1_053,7.814837,12.020758,7.439361,7.424397,6.194262,6.749798,7.243300,7.036282,10.808758,...,10.249087,4.263861,3.957766,3.512115,4.083341,5.456798,5.451782,8.790905,4.292198,3.628460
3,HOME_&_GARDEN_1_140,4.652560,5.025778,4.002103,3.569119,2.675298,3.133356,3.104004,5.309199,5.181020,...,5.180179,3.085345,3.454551,2.920507,2.848674,3.146972,4.465405,5.180179,3.085345,3.454551
4,HOME_&_GARDEN_1_177,5.669504,5.836857,3.855762,4.415171,4.004750,3.640381,3.044561,5.001417,5.705845,...,5.734345,3.647694,4.032104,4.453071,3.624151,3.060585,4.770714,5.734345,3.647694,4.032104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SUPERMARKET_3_499,12.305567,10.023282,12.284995,5.778226,7.055702,10.910690,6.949996,11.366982,5.863937,...,3.952742,9.915113,4.412012,8.595744,5.472510,9.977044,7.260520,4.501563,9.602042,4.573679
96,ACCESORIES_1_158,5.519582,6.620786,5.095683,4.566849,4.226579,5.329988,2.957885,5.190460,4.164193,...,4.549639,3.033232,3.578396,2.909196,3.600921,3.678135,4.451833,4.278294,3.462548,3.445595
97,SUPERMARKET_3_282,55.423673,71.586254,57.270722,50.600475,45.183500,40.435065,40.124135,53.150632,65.510171,...,65.881627,51.883173,49.710599,41.269786,38.228984,35.285020,51.112119,65.386151,51.003344,51.153575
98,ACCESORIES_1_354,4.417852,9.111879,6.381964,5.877536,5.222049,5.770861,5.843167,5.302681,6.764190,...,7.640582,6.716417,6.874929,6.667891,7.056631,6.670139,7.373004,7.988651,6.858462,6.210917


In [8]:
#df_resultados = df_resultados.apply(np.ceil)

# Reorganizar el DataFrame para tener una columna 'dia' y una columna 'prediccion'
df_resultados_long = pd.melt(df_resultados, id_vars=['producto'], 
                             var_name='dia', value_name='prediccion')

# Mostrar el DataFrame resultante con las predicciones en una sola columna
print(df_resultados_long)

                 producto    dia  prediccion
0        ACCESORIES_1_108   dia1    1.452962
1     HOME_&_GARDEN_1_027   dia1    4.474940
2     HOME_&_GARDEN_1_053   dia1    7.814837
3     HOME_&_GARDEN_1_140   dia1    4.652560
4     HOME_&_GARDEN_1_177   dia1    5.669504
...                   ...    ...         ...
5995    SUPERMARKET_3_499  dia60    4.573679
5996     ACCESORIES_1_158  dia60    3.445595
5997    SUPERMARKET_3_282  dia60   51.153575
5998     ACCESORIES_1_354  dia60    6.210917
5999  HOME_&_GARDEN_1_106  dia60   12.992893

[6000 rows x 3 columns]


In [9]:
df_resultados_long.sort_values('producto')

,producto,dia,prediccion
5662,ACCESORIES_1_064,dia57,1.841934
2662,ACCESORIES_1_064,dia27,2.261380
1262,ACCESORIES_1_064,dia13,1.513202
5762,ACCESORIES_1_064,dia58,2.802833
5162,ACCESORIES_1_064,dia52,2.468901
...,...,...,...
5485,SUPERMARKET_3_811,dia55,15.034952
2485,SUPERMARKET_3_811,dia25,12.952321
1385,SUPERMARKET_3_811,dia14,14.225262
1785,SUPERMARKET_3_811,dia18,13.500571


In [3]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'BOS_3') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'sales', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100),
        lags=14
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)

    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    # Crear un rango de fechas
    date_range = pd.date_range(start=start_date, end=end_date)
    # Crear el dataframe
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)
    
    # Predecir
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    #print(f"Predicciones generadas para el producto: {producto}")


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 